In [5]:
import pandas as pd

df = pd.read_parquet('/drive2/tuandung/DevEval/Generations/ds67/LFCLF_embedding_dev_eval_deepseek-ai_deepseek-coder-6.7b-instruct_1_clean_code.parquet')

In [6]:
df

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,cleaned_code
0,benedict.utils.type_util.is_json_serializable,benedict.utils.type_util.is_json_serializable_0,281,```Python\ndef is_json_serializable(val):\n ...,"[-0.103515625, 0.447265625, -0.1884765625, -0....","[-0.053466796875, 0.08251953125, -0.0157470703...","[0.0068359375, 0.173828125, -0.0908203125, -0....","[0.2412109375, -0.30078125, 0.2119140625, -0.2...",False,def is_json_serializable(val):\n if is_bool...,if is_bool(val) or is_integer(val) or is_f...
1,benedict.utils.type_util.is_json_serializable,benedict.utils.type_util.is_json_serializable_1,348,```Python\ndef is_json_serializable(val):\n ...,"[-0.103515625, 0.447265625, -0.1884765625, -0....","[-0.068359375, 0.0380859375, 0.002822875976562...","[0.0068359375, 0.173828125, -0.0908203125, -0....","[0.2412109375, -0.296875, 0.21484375, -0.26171...",False,def is_json_serializable(val):\n if is_bool...,if is_bool(val) or is_integer(val) or is_f...
2,benedict.utils.type_util.is_json_serializable,benedict.utils.type_util.is_json_serializable_2,163,```Python\ndef is_json_serializable(val):\n ...,"[-0.103515625, 0.447265625, -0.1884765625, -0....","[-0.005126953125, 0.515625, 0.06640625, -0.034...","[0.0068359375, 0.173828125, -0.0908203125, -0....","[0.2265625, -0.1630859375, -0.0380859375, 0.08...",False,def is_json_serializable(val):\n if is_bool...,if is_bool(val) or is_integer(val) or is_f...
3,benedict.utils.type_util.is_json_serializable,benedict.utils.type_util.is_json_serializable_3,115,```Python\ndef is_json_serializable(val):\n ...,"[-0.103515625, 0.447265625, -0.1884765625, -0....","[-0.018310546875, 0.5234375, 0.078125, -0.0322...","[0.0068359375, 0.173828125, -0.0908203125, -0....","[0.2275390625, -0.15234375, -0.02392578125, 0....",False,def is_json_serializable(val):\n if isinsta...,"if isinstance(val, (str, int, float, bool,..."
4,benedict.utils.type_util.is_json_serializable,benedict.utils.type_util.is_json_serializable_4,175,```Python\ndef is_json_serializable(val):\n ...,"[-0.103515625, 0.447265625, -0.1884765625, -0....","[-0.003662109375, 0.51171875, 0.060546875, -0....","[0.0068359375, 0.173828125, -0.0908203125, -0....","[0.220703125, -0.171875, -0.0458984375, 0.0839...",False,def is_json_serializable(val):\n if is_bool...,if is_bool(val) or is_integer(val) or is_f...
...,...,...,...,...,...,...,...,...,...,...,...
17325,praw.util.token_manager.SQLiteTokenManager.reg...,praw.util.token_manager.SQLiteTokenManager.reg...,110,"```Python\n def register(self, refresh_toke...","[-0.1025390625, 0.45703125, -0.1904296875, -0....","[-0.05859375, 0.494140625, 0.04345703125, -0.0...","[-0.0498046875, 0.10205078125, -0.08837890625,...","[0.1650390625, -0.2041015625, -0.045166015625,...",False,"def register(self, refresh_token):\n ""...",if self.is_registered():\n ...
17326,praw.util.token_manager.SQLiteTokenManager.reg...,praw.util.token_manager.SQLiteTokenManager.reg...,110,"```Python\n def register(self, refresh_toke...","[-0.1025390625, 0.45703125, -0.1904296875, -0....","[-0.05859375, 0.494140625, 0.04345703125, -0.0...","[-0.0498046875, 0.10205078125, -0.08837890625,...","[0.1650390625, -0.2041015625, -0.045166015625,...",False,"def register(self, refresh_token):\n ""...",if self.is_registered():\n ...
17327,praw.util.token_manager.SQLiteTokenManager.reg...,praw.util.token_manager.SQLiteTokenManager.reg...,114,"```Python\n def register(self, refresh_toke...","[-0.1025390625, 0.45703125, -0.1904296875, -0....","[-0.053466796875, 0.486328125, 0.046875, -0.07...","[-0.0498046875, 0.10205078125, -0.08837890625,...","[0.1611328125, -0.2001953125, -0.037353515625,...",False,"def register(self, refresh_token):\n ""...",if self.is_registered():\n ...
17328,praw.util.token_manager.SQLiteTokenManager.reg...,praw.util.token_manager.SQLiteTokenManager.reg...,160,"```Python\n def register(self, refresh

In [3]:
import json

fail_namespaces = []

with open('/drive2/tuandung/DevEval/failed_samples.jsonl') as f:
    for line in f:
        data = json.loads(line)
        fail_namespaces.append(data['namespace'])

In [5]:
datasets = []
with open('/drive2/tuandung/DevEval/data.jsonl') as f:
    for line in f:
        data = json.loads(line)
        if data['namespace'] not in fail_namespaces:
            datasets.append(data)
len(datasets)

1733

In [6]:
with open('/drive2/tuandung/DevEval/data_clean.jsonl', 'w') as f:
    for data in datasets:
        f.write(json.dumps(data) + '\n')

In [5]:
import json

benchmark_data = []
project_data = {}
with open('/drive2/tuandung/DevEval/data.jsonl', 'r') as f:
    for line in f:
        js = json.loads(line)
        benchmark_data.append(js)
        project_name = js['completion_path'].split('/')[0]
        if project_name not in project_data:
            project_data[project_name] = []
        project_data[project_name].append(js)

import os
source_code_root = '/drive2/tuandung/DevEval/Source_Code'

ground_truth_completions = []
for data in benchmark_data:
    sos, eos = data['body_position'][0]-1, data['body_position'][1]
    completion_path = os.path.join(source_code_root, data['completion_path'])
    with open(completion_path, 'r') as f:
        file_lines = f.readlines()
    # file_lines = file_lines[:sos] + ['\n', completion, '\n'] + file_lines[eos:]
    ground_truth = ''.join(file_lines[sos:eos])
    ground_truth_completions.append({
        'namespace': data['namespace'],
        'completion': ground_truth  
    })

with open('ground_truth_completions.jsonl', 'w') as f:
    for js in ground_truth_completions:
        f.write(json.dumps(js) + '\n')